In [29]:
!pip install -q google-genai google-adk

In [30]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [31]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

In [32]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

In [33]:
# Cria um serviço de sessão em memória

session_service = InMemorySessionService()

In [34]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

async def call_agent(agent: Agent, message_text: str) -> str:
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [35]:
# Lista todos os modelos disponíveis atualmente

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/ge

In [43]:
MODELO = "gemini-2.0-flash"

In [37]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model=MODELO, # Usando o modelo gratuito
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [38]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################

async def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODELO, # Usando o modelo gratuito
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais.
        Você recebe uma lista de lançamentos recentes do agente buscador e,
        - Para cada um dos lançamentos recebidos, você deve usar a ferramenta
        de busca do Google (google_search) para buscar os pontos mais relevantes
        que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais
        informações sobre cada um dos temas e aprofundar.
        - Depois de terminar a busca, você irá escolher APENAS UM tema dentre todos eles,
        aquele tema com potencial de ser o mais relevante com base nas suas pesquisas.
        - Depois de escolher o tema mais relevante dentre todos eles, você irá retornar
        qual foi o tema escolhido, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = await call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [39]:
######################################
# --- Agente 3: Redator do Post --- #
######################################

async def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODELO,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"

    # Executa o agente
    post_final = await call_agent(redator, entrada_do_agente_redator)
    return post_final

In [40]:
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [41]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

In [44]:
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀\n")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    post_redigido = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(post_redigido))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Inteligência Artificial

Maravilha! Vamos então criar o post sobre novidades em Inteligência Artificial

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes em Inteligência Artificial no último mês, vou realizar algumas pesquisas no Google. Eis as minhas questões e palavras-chave de busca:
> 
> 
> Com base nas notícias e tendências recentes em inteligência artificial, aqui estão alguns dos lançamentos e avanços mais relevantes até agosto de 2025:
> 
> 1.  **GPT-5:** A OpenAI lançou a quinta geração da sua tecnologia de inteligência artificial, o GPT-5. Apesar de algumas controvérsias iniciais sobre o seu desempenho, o GPT-5 apresenta avanços significativos, incluindo custos de acesso via API 50% menores, tornando-o mais acessível para desenvolvedores e empresas. Ele também integra novas ferramentas de busca que permitem consultar informações na internet em tempo real. Sam Altman, CEO da OpenAI, descreveu os testes do novo modelo como um "momento 'é isso'", indicando um avanço substancial.
> 2.  **Adoção de IA Generativa:** Mais de 70% das organizações já experimentaram IA generativa, impulsionadas pela procura de eficiência, inovação e novos modelos de negócio. Um estudo da Devoteam revelou que 81 das 150 tecnologias analisadas já integram IA, o que indica uma mudança estrutural na forma como as empresas inovam e escalam as suas operações. O crescimento das plataformas "agentic-ready", preparadas para responder em tempo real e automatizar fluxos com base em IA, também é notável.
> 3.  **Meta AI:** A Meta lançou um aplicativo próprio para a Meta AI no Brasil, desenvolvido com o Llama. A inteligência artificial da Meta já estava disponível no WhatsApp, Instagram, Facebook e Messenger, mas agora se torna um assistente pessoal de IA em um app independente. O aplicativo oferece um feed "Descobrir" para explorar como outras pessoas estão usando a IA e utiliza o modelo multimodal Llama 4 para resolver problemas, responder dúvidas e oferecer recomendações.
> 4.  **Supercomputadores para IA:** O Governo de Goiás, no Brasil, sedia os três primeiros supercomputadores Nvidia DGX B200 da América Latina, marcando um compromisso com a modernização da gestão pública através da tecnologia. A parceria entre o Centro de Excelência em Inteligência Artificial da Universidade Federal de Goiás (CEIA/UFG) e o governo estadual visa fomentar a pesquisa e o desenvolvimento da IA, com potencial para beneficiar áreas como saúde, educação, controle e economia.
> 5.  **Tendências de IA para 2025:** A inteligência artificial continua a ser um tema central para as empresas, que a adotam para otimizar operações, reduzir custos e atender às exigências dos clientes. Tendências importantes incluem a automação inteligente de processos, a hiperpersonalização das interações com clientes e fornecedores e a IA colaborativa (Human-in-the-Loop), onde a IA e os humanos trabalham juntos para melhorar os resultados.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nos lançamentos e avanços recentes em inteligência artificial, vou analisar cada um deles para identificar o tema mais relevante para um post nas redes sociais.
> 
> **1. GPT-5:**
> 
> *   **Pontos Relevantes:** Lançamento da nova geração do GPT, redução de custos via API, integração de ferramentas de busca em tempo real, declarações impactantes do CEO da OpenAI.
> *   **Pesquisa Adicional:** Buscar detalhes sobre as novas ferramentas de busca integradas, casos de uso iniciais do GPT-5, comparações de desempenho com o GPT-4, reações da comunidade de desenvolvedores.
>     
> **2. Adoção de IA Generativa:**
> 
> *   **Pontos Relevantes:** Alta taxa de adoção de IA generativa em organizações, busca por eficiência e inovação, integração de IA em diversas tecnologias, crescimento de plataformas "agentic-ready".
> *   **Pesquisa Adicional:** Exemplos de casos de uso de IA generativa em empresas, benefícios específicos relatados pelas organizações que adotaram a IA, desafios na implementação da IA generativa.
>     
> **3. Meta AI:**
> 
> *   **Pontos Relevantes:** Lançamento de aplicativo próprio da Meta AI no Brasil, uso do Llama 4, assistente pessoal de IA independente, feed "Descobrir", modelo multimodal para resolver problemas e oferecer recomendações.
> *   **Pesquisa Adicional:** Detalhes sobre o modelo Llama 4, funcionalidades específicas do aplicativo Meta AI, exemplos de uso do feed "Descobrir", impacto do lançamento no mercado brasileiro.
>     
> **4. Supercomputadores para IA:**
> 
> *   **Pontos Relevantes:** Instalação dos primeiros supercomputadores Nvidia DGX B200 na América Latina em Goiás, parceria entre CEIA/UFG e governo estadual, foco em pesquisa e desenvolvimento de IA, potencial para beneficiar áreas como saúde, educação, controle e economia.
> *   **Pesquisa Adicional:** Detalhes técnicos dos supercomputadores Nvidia DGX B200, projetos específicos de pesquisa em IA que serão realizados em Goiás, impacto esperado nas áreas de saúde, educação, controle e economia.
>     
> **5. Tendências de IA para 2025:**
> 
> *   **Pontos Relevantes:** Otimização de operações, redução de custos, atendimento às exigências dos clientes, automação inteligente de processos, hiperpersonalização das interações, IA colaborativa (Human-in-the-Loop).
> *   **Pesquisa Adicional:** Exemplos de automação inteligente de processos, casos de hiperpersonalização bem-sucedidos, modelos de IA colaborativa e seus resultados, previsões de especialistas sobre a evolução dessas tendências.
>     
> Após analisar todos os lançamentos e tendências, o tema mais relevante para um post nas redes sociais é o **GPT-5**. Ele gera o maior impacto e interesse, e possui vários pontos que podem ser explorados em um post.
> 
> **Pontos Relevantes:**
> 
> *   Lançamento da nova geração do GPT, com melhorias significativas em relação ao GPT-4.
> *   Redução de custos via API, tornando-o mais acessível para desenvolvedores e empresas.
> *   Integração de novas ferramentas de busca que permitem consultar informações na internet em tempo real.
> *   Declarações impactantes do CEO da OpenAI, Sam Altman, descrevendo os testes do novo modelo como um "momento 'é isso'".
> *   Melhor desempenho em diversas áreas, como codificação, escrita, raciocínio e redução de alucinações.
> 
> **Plano para o Post:**
> 
> 1.  **Título:** GPT-5 Chegou: O Que Mudou e Como Você Pode Usar a Nova IA da OpenAI
> 2.  **Introdução:** Apresentar o GPT-5 como a mais recente e poderosa ferramenta de IA da OpenAI, destacando o impacto do lançamento e o entusiasmo gerado na comunidade de tecnologia.
> 3.  **Novidades e Melhorias:**
>     *   Comparar o GPT-5 com o GPT-4, detalhando as melhorias em desempenho, precisão, velocidade e capacidade de raciocínio.
>     *   Explicar as novas ferramentas de busca integradas e como elas permitem o acesso a informações em tempo real.
>     *   Mencionar a redução de custos via API e o impacto na acessibilidade para desenvolvedores e empresas.
>     *   Destacar a capacidade do GPT-5 de "alucinar menos" e fornecer respostas mais confiáveis.
> 4.  **Aplicações e Casos de Uso:**
>     *   Apresentar exemplos de como o GPT-5 pode ser usado em diferentes áreas, como codificação, criação de conteúdo, suporte ao cliente e pesquisa.
>     *   Compartilhar casos de uso específicos, como a criação de jogos, a correção de bugs em SDKs e a otimização de fluxos de trabalho.
> 5.  **Como Acessar e Usar o GPT-5:**
>     *   Informar que o GPT-5 já está disponível para usuários gratuitos e pagos do ChatGPT.
>     *   Explicar como acessar o GPT-5 via API e as diferentes versões disponíveis para desenvolvedores (GPT-5, GPT-5 Mini e GPT-5 Nano).
>     *   Incentivar os usuários a experimentarem o GPT-5 e compartilharem suas experiências nas redes sociais.
> 6.  **Conclusão:** Reafirmar o potencial do GPT-5 como uma ferramenta transformadora para diversas áreas e destacar a importância de acompanhar os avanços da IA para se manter atualizado e competitivo.
> 
> Este plano permite criar um post informativo, interessante e relevante para o público, incentivando a interação e o compartilhamento de informações sobre o GPT-5.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚀 GPT-5 CHEGOU: Prepare-se para a revolução da IA! 🚀
> 
> A OpenAI acaba de lançar o GPT-5, e a gente tá aqui pra te contar TUDO! 🤩
> 
> 🔥 **O que mudou?**
> *   Performance turbinada 🚀: Mais rápido, mais preciso e com raciocínio aprimorado.
> *   Busca integrada 🌐: Informações em tempo real na ponta dos seus dedos.
> *   API mais acessível 💰: Desenvolvedores e empresas, preparem-se para inovar!
> *   Menos "alucinações" 🤔: Respostas mais confiáveis e relevantes.
> 
> 💡 **Onde usar?**
> *   Codificação 💻: Crie, teste e refine seus códigos com a ajuda da IA.
> *   Conteúdo criativo ✍️: Deixe o GPT-5 turbinar sua imaginação!
> *   Suporte ao cliente 🤝: Atendimento eficiente e personalizado.
> *   Pesquisa e desenvolvimento 🔬: Acelere suas descobertas com a IA.
> 
> ✨ **Como experimentar?**
> O GPT-5 já está disponível para usuários ChatGPT! 😉 Acesse via API e explore as versões GPT-5, GPT-5 Mini e GPT-5 Nano.
> 
> 👉 **Experimente o GPT-5 e compartilhe suas experiências!** Use a hashtag #GPT5Alura e mostre pra gente o que você está criando!
> 
> A inteligência artificial está transformando o mundo, e o GPT-5 é mais um passo nessa jornada. 🚀 Não fique de fora! 😉
> 
> #InteligenciaArtificial #OpenAI #Alura


--------------------------------------------------------------
